In [2]:
import os
from datetime import datetime
from pathlib import Path

import pandas as pd
import numpy as np

import dask
from dask import dataframe as dd
from dask import array as da

import polars as pl

data_path = Path(os.getcwd()).parent/"data"

In [14]:
pf = pl.read_csv(data_path/"credit_card_transactions-ibm_v2.csv")

In [16]:
pf.describe()

statistic,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
str,f64,f64,f64,f64,f64,str,str,str,f64,str,str,f64,f64,str,str
"""count""",2.43869e7,2.43869e7,2.43869e7,2.43869e7,2.43869e7,"""24386900""","""24386900""","""24386900""",2.43869e7,"""24386900""","""21666079""",2.1508765e7,2.43869e7,"""388431""","""24386900"""
"""null_count""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0""",0.0,"""0""","""2720821""",2.878135e6,0.0,"""23998469""","""0"""
"""mean""",1001.019335,1.351366,2011.95517,6.525064,15.718123,null,null,null,-4.7692e17,null,null,50956.442115,5561.171253,null,null
"""std""",569.461157,1.407154,5.105921,3.472355,8.794073,null,null,null,4.7589e18,null,null,29397.065949,879.315433,null,null
"""min""",0.0,0.0,1991.0,1.0,1.0,"""00:00""","""$-0.00""","""Chip Transaction""",-9.2229e18,"""Aaronsburg""","""AA""",501.0,1711.0,"""Bad CVV""","""No"""
"""25%""",510.0,0.0,2008.0,3.0,8.0,null,null,null,-4.5005e18,null,null,28374.0,5300.0,null,null
"""50%""",1006.0,1.0,2013.0,7.0,16.0,null,null,null,-7.9468e17,null,null,46742.0,5499.0,null,null
"""75%""",1477.0,2.0,2016.0,10.0,23.0,null,null,null,3.1895e18,null,null,77564.0,5812.0,null,null
"""max""",1999.0,8.0,2020.0,12.0,31.0,"""23:59""","""$999.97""","""Swipe Transaction""",9.2233e18,"""Zwolle""","""Zimbabwe""",99928.0,9402.0,"""Technical Glitch""","""Yes"""
